In [44]:
import pathlib

from Bio import PDB
from loguru import logger
from openmm.app import PDBxFile
from pdbfixer import PDBFixer
from tqdm.auto import tqdm

In [25]:
PROJECT_ROOT = pathlib.Path().resolve().parent
TEST_PDB_ID_LIST = ["3idp", "6X18", "8RX0"]

In [ ]:
# Download PDB files
pdb_list = PDB.PDBList(server='https://files.wwpdb.org')
for id_ in TEST_PDB_ID_LIST:
    pdb_list.retrieve_pdb_file(pdb_code=id_, pdir=PROJECT_ROOT / "data")

Structure exists: 'C:\Users\janoj\GitHub\LoopBuilder\data\3idp.cif' 


## PDBFixer

Run the loop building multiple times (with different random seeds; non-controllable) to generate an ensemble of structures.

In [47]:
input_file = PROJECT_ROOT / f"data/3idp.cif"

for i in tqdm(range(1, 5)):
    fixer = PDBFixer(filename=str(input_file))
    fixer.findMissingResidues()

    chains = list(fixer.topology.chains())
    non_terminal = {}
    for key, value in fixer.missingResidues.items():
        chain_index, missing_index = key
        if missing_index == 0 :
            continue

        if missing_index == len(chains[chain_index]):
            continue

        non_terminal[key] = value

    fixer.missingResidues = non_terminal

    fixer.findMissingAtoms()
    fixer.addMissingAtoms()

    output_dir = PROJECT_ROOT / "sandbox/PDBFixer"
    with open(output_dir / f"{input_file.stem}_fixed_api_{i}.cif", 'w') as fp:
        PDBxFile.writeFile(fixer.topology, fixer.positions, fp)

  0%|          | 0/4 [00:00<?, ?it/s]